<a href="https://colab.research.google.com/github/gorovuha/nlp/blob/main/BERT_multi-classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!pip install datasets

Из данного датасета для обучения понадобятся колонки 'utt' и 'intent'. Для задачи мультиклассовой классификации удобно использовать структуру: признак (intent), значения признака (60 интентов) и для каждого признака True/False, чтобы знать, с каким признаком сопоставим данный пример. Такая структура также подойдёт, если необходимо отнести пример к нескольким интентам.
Для этого выбираем часть из Amazon massive и записываем в виде датасета.


In [ ]:
from datasets import load_dataset
from datasets import Dataset


dataset = load_dataset("AmazonScience/massive", "en-US")
dataset_train = {'id': [], 'utt': []}
dataset_valid = {'id': [], 'utt': []}

for i in range(60):
  dataset_train[i] = []
  dataset_valid[i] = []


print(dataset_train[4])
print(len(dataset['train']['intent']))
for i in range(len(dataset['train']['intent'])):
  num_int_train = dataset['train']['intent'][i]

  dataset_train[num_int_train].append(True)

  for g in range(60):
    if g != num_int_train:
       dataset_train[g].append(False)
  #print(dataset_train)
  dataset_train['utt'].append(dataset['train']['utt'][i])

  dataset_train['id'].append(dataset['train']['id'][i])

  #print(i)

for i in range(len(dataset['validation']['intent'])):
  num_int_valid = dataset['validation']['intent'][i]

  dataset_valid[num_int_valid].append(True)

  for g in range(60):
    if g != num_int_valid:
       dataset_valid[g].append(False)
  #print(dataset_train)
  dataset_valid['utt'].append(dataset['validation']['utt'][i])

  dataset_valid['id'].append(dataset['validation']['id'][i])

print(dataset_train)

In [ ]:
Из полученного словаря на шаге выше формируем dataset

In [89]:
from datasets import Dataset
import pandas as pd

data = {}
data['train'] = Dataset.from_pandas(pd.DataFrame(data=dataset_train))
data['validation'] = Dataset.from_pandas(pd.DataFrame(data=dataset_valid))
print(data)


/usr/local/lib/python3.10/dist-packages/datasets/table.py:761: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  return cls(pa.Table.from_pandas(*args, **kwargs))
/usr/local/lib/python3.10/dist-packages/datasets/table.py:761: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  return cls(pa.Table.from_pandas(*args, **kwargs))


{'train': Dataset({
    features: ['id', 'utt', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59'],
    num_rows: 11514
}), 'validation': Dataset({
    features: ['id', 'utt', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59'],
    num_rows: 2033
})}


Dataset({
    features: ['id', 'utt', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59'],
    num_rows: 11514
})

In [ ]:
Номера интентов 0-59 будут labels для обучения модели.

In [ ]:
labels = [label for label in data['train'].features.keys() if label not in ['id', 'utt']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

In [ ]:
Для решения задачи мультиклассовой классификации отлично подойдёт трансформер BERT, для которого мы укажем тип задачи problem_type и колчество признаков (интентов)

In [103]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
!pip install datasets transformers==4.27.4

In [ ]:
Предобработка данных. Предложения 'utt' нужно токенизировать и декодировать.

In [104]:
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def preprocess_data(examples):
  # take a batch of texts
  text = examples["utt"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [ ]:
Выполняем предобработку с помощью описанной выше функции.

In [105]:
import numpy as np

encoded_dataset = data['train'].map(preprocess_data, batched=True, remove_columns = data['train'].column_names)
encoded_dataset_validation = data['validation'].map(preprocess_data, batched = True, remove_columns = data['validation'].column_names)

Map:   0%|          | 0/11514 [00:00<?, ? examples/s]

Map:   0%|          | 0/2033 [00:00<?, ? examples/s]

In [ ]:
Вот так выглядит токенизированное предложение.

In [95]:
tokenizer.decode(example['input_ids'])

'[CLS] wake me up at nine am on friday [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
Проверяем, что всё работает

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['48']

In [ ]:
Переводим наш датасет в тензор, используем PyTorch.

In [96]:
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [97]:
batch_size = 8
metric_name = "f1"

In [ ]:
Параметры для обучения, выбраны такие, чтобы получить максимальную точность (accuracy)

In [109]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_steps=100,
    save_total_limit=2,
    #push_to_hub=True,
)

In [ ]:
Процедуры для подсчёта метрик, на основе который будет оценена модель

In [110]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
Создаём экземпляр класса Trainer с нашими args

In [111]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Обучение модели. Accuracy 87.7%





In [112]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.068900,0.065216,0.084666,0.522122,0.044270
2,0.038200,0.033222,0.654176,0.752962,0.506640
3,0.021300,0.021652,0.834498,0.881381,0.763896
4,0.013700,0.017852,0.872379,0.913808,0.828824
5,0.009200,0.015793,0.884830,0.928235,0.856862
6,0.007600,0.015223,0.884528,0.931599,0.863748
7,0.005600,0.015205,0.886887,0.935013,0.869159
8,0.004900,0.014623,0.891158,0.939186,0.877029
9,0.003700,0.014648,0.893819,0.940199,0.878505
10,0.003800,0.014712,0.892492,0.939211,0.877521


TrainOutput(global_step=14400, training_loss=0.023342231774909628, metrics={'train_runtime': 3193.6506, 'train_samples_per_second': 36.053, 'train_steps_per_second': 4.509, 'total_flos': 7577595772784640.0, 'train_loss': 0.023342231774909628, 'epoch': 10.0})

Пробуем предсказать значение интента для случайной фразы

In [127]:
text = "Set alarm for 7 am"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [128]:
logits = outputs.logits
logits.shape

torch.Size([1, 60])

Возвращаем значение интента

In [129]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1

predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['48']


In [116]:
model.save_pretrained("path/to/lana")